In [1]:
from emg_fatigue.utils.load_emg_data import load_all_participant_data
from emg_fatigue.utils.process_emg_data import process_all_participant_data
from emg_fatigue.utils.create_loocv_dataset import create_loocv_dataset
from emg_fatigue.modeling.train import train_model
from emg_fatigue.modeling.build.rnn_model import build_lstm_model
from emg_fatigue.plots.visualize_model_predictions import visualize_model_predictions
from emg_fatigue.analysis.slope_analysis import analyze_label_slopes, print_loocv_results_table
from emg_fatigue.config import BATCH_SIZE, PADDING_VALUE

2025-04-10 20:12:39.186427: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-10 20:12:39.187163: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-10 20:12:39.191477: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-10 20:12:39.201497: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744308759.214945  605290 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744308759.21

In [2]:
raw_data = load_all_participant_data()
processed_data = process_all_participant_data(raw_data)

analysis_results = analyze_label_slopes(processed_data)
if analysis_results:
    print_loocv_results_table(analysis_results)

# --- Create LOO-CV Datasets ---
all_participant_ids = list(processed_data.keys())
train_ids = all_participant_ids[:-3] 
val_ids = all_participant_ids[-3:-1]
test_ids = [all_participant_ids[-1]]

train_ds, val_ds, test_ds, input_shape, output_shape = create_loocv_dataset(
    processed_data,
    train_participant_ids=train_ids,
    validation_participant_ids=val_ids,
    test_participant_ids=test_ids,
    batch_size=BATCH_SIZE,
    padding_value=PADDING_VALUE
)



2025-04-10 20:12:46.470 | INFO     | emg_fatigue.analysis.slope_analysis:analyze_label_slopes:40 - Analyzing true fatigue label slopes across all recordings...
2025-04-10 20:12:46.482 | INFO     | emg_fatigue.analysis.slope_analysis:analyze_label_slopes:89 - Overall True Label Slope Stats (n=192): Mean=1.2094, Median=1.1801, Std=0.2289
2025-04-10 20:12:46.482 | INFO     | emg_fatigue.analysis.slope_analysis:analyze_label_slopes:96 - Performing leave-one-out evaluation of baseline linear models...
2025-04-10 20:12:46.483 | INFO     | emg_fatigue.analysis.slope_analysis:analyze_label_slopes:168 - Participant P001 - MAE using leave-one-out mean slope: 8.5406, MAE using leave-one-out median slope: 7.8679
2025-04-10 20:12:46.483 | INFO     | emg_fatigue.analysis.slope_analysis:analyze_label_slopes:168 - Participant P002 - MAE using leave-one-out mean slope: 5.4344, MAE using leave-one-out median slope: 7.0823
2025-04-10 20:12:46.484 | INFO     | emg_fatigue.analysis.slope_analysis:analyze_l

2025-04-10 20:12:46.544410: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [3]:
lstm_model = build_lstm_model(input_shape, output_shape, padding_value=PADDING_VALUE)

train_model(lstm_model, train_ds, val_ds, "lstm_fatigue_model", epochs=100)

2025-04-10 20:12:46.772 | INFO     | emg_fatigue.modeling.train:train_model:48 - Model will be saved to: /home/lg519/Projects/Muscle_Fatigue_Research/emg_fatigue/models/lstm_fatigue_model.keras
2025-04-10 20:12:46.772 | INFO     | emg_fatigue.modeling.train:train_model:77 - Starting model training for up to 100 epochs...
Epoch 1/100


/home/lg519/.cache/pypoetry/virtualenvs/emg-fatigue-YbMeuwX9-py3.12/lib/python3.12/site-packages/keras/src/layers/layer.py:940: UserWarning: Layer 'squeeze_output' (of type Lambda) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(
2025-04-10 20:12:58.196106: E tensorflow/core/util/util.cc:131] oneDNN supports DT_BOOL only on platforms with AVX-512. Falling back to the default Eigen-based implementation if present.


5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 512ms/step - loss: 2112.9429 - mae: 32.6172
Epoch 1: val_loss improved from inf to 2008.84424, saving model to /home/lg519/Projects/Muscle_Fatigue_Research/emg_fatigue/models/lstm_fatigue_model.keras
5/5 ━━━━━━━━━━━━━━━━━━━━ 18s 1s/step - loss: 2107.6655 - mae: 32.6420 - val_loss: 2008.8442 - val_mae: 34.4617 - learning_rate: 0.0010
Epoch 2/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 533ms/step - loss: 1681.7729 - mae: 30.6389
Epoch 2: val_loss improved from 2008.84424 to 1593.41919, saving model to /home/lg519/Projects/Muscle_Fatigue_Research/emg_fatigue/models/lstm_fatigue_model.keras
5/5 ━━━━━━━━━━━━━━━━━━━━ 3s 590ms/step - loss: 1676.6299 - mae: 30.6565 - val_loss: 1593.4192 - val_mae: 31.7844 - learning_rate: 0.0010
Epoch 3/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 524ms/step - loss: 1379.7649 - mae: 29.5965
Epoch 3: val_loss improved from 1593.41919 to 1325.32788, saving model to /home/lg519/Projects/Muscle_Fatigue_Research/emg_fatigue/models/lstm_fatigue_model.keras
5/5 ━━

In [4]:
visualize_model_predictions(
    model=lstm_model,
    processed_data=processed_data,
    test_participant_ids=test_ids,
    input_shape=input_shape
)

2025-04-10 20:17:21.421 | INFO     | emg_fatigue.plots.visualize_model_predictions:visualize_model_predictions:59 - Starting prediction visualization for 1 test participants.
2025-04-10 20:17:21.422 | INFO     | emg_fatigue.plots.visualize_model_predictions:visualize_model_predictions:70 - Visualizing predictions for participant P010...
2025-04-10 20:17:21.422 | INFO     | emg_fatigue.plots.visualize_model_predictions:visualize_model_predictions:91 -   Processing recording 1/8 for P010 left...
2025-04-10 20:17:24.635 | INFO     | emg_fatigue.plots.visualize_model_predictions:visualize_model_predictions:245 -   Recording 1 MAEs - Overall: 8.40, Before 50s: 3.73, After 50s: 15.24
2025-04-10 20:17:26.296 | INFO     | emg_fatigue.plots.visualize_model_predictions:visualize_model_predictions:401 -   Saved prediction plot to: /home/lg519/Projects/Muscle_Fatigue_Research/emg_fatigue/reports/figures/predictions_P010_left_rec1.png
2025-04-10 20:17:26.297 | INFO     | emg_fatigue.plots.visualize